In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from urllib.parse import urljoin

In [2]:
def selenium_set():

    global driver
    
    # chrome path setting
    driver_path = "/workspace/chromedriver-linux64/chromedriver"
    chrome_path = "/workspace/chrome-linux64/chrome"
    
    chrome_options = webdriver.ChromeOptions()
    
    chrome_options.binary_location = chrome_path
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    service = Service(executable_path=driver_path)
    
    driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
#topic별 url 리스트 반환
def make_topic_urllist(data, base_url):
    urllist=[]
    topic_txt_lnb=pd.read_csv(data, encoding='utf-8')
    for row in topic_txt_lnb.iloc[:,1]:
        url = base_url + str(row)
        urllist.append(url) 
    return urllist

In [4]:
def scroll_click_extract(url):

    """
    - 페이지 바닥까지 스크롤하면서 '기사 더보기' 버튼을 반복 클릭
    - 반복이 끝나면 현재 페이지의 모든 기사 링크를 수집하여 DataFrame 반환
    """
    
    global driver
    driver.get(url)

    try:
        response = requests.get(url)
        response.raise_for_status()  # 문제가 발생하면 예외 발생
        html = response.text
    except requests.exceptions.RequestException as e:
        print(f"웹 페이지 요청 오류: {e}")
        exit()

    
    # XPath로 '기사 더보기' 버튼 찾기
    refresh_btn = driver.find_element(By.XPATH, '//a[contains(@class, "_CONTENT_LIST_LOAD_MORE_BUTTON")]')

    count = 0
    
    while count < 0: ############ 클릭 끝까지로 바꾸기 ############
        count += 1
        print(f"[LOOP] {count} 번째 반복 중...")
        
        try:
            # 페이지 끝까지 스크롤하기
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            for _ in range(10):  # 10번 정도 END 키 누르기
                body.send_keys(Keys.END)

            # 클릭
            # driver.execute_script("arguments[0].click();", refresh_btn)
            # time.sleep(1)  # 새 기사 로딩 대기

            # 최대 5초간 '기사 더보기' 버튼이 클릭 가능해질 때까지 기다림
            button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(@class, "_CONTENT_LIST_LOAD_MORE_BUTTON")]'))
            )
            button.click()
            print(f" → 버튼 클릭 성공 ({count}회)")
                        
        except Exception as e:
            print("기사 더보기 버튼 클릭 실패:", e)
            break

    """
    --- 불러온 HTMl에서 기사 링크 모두 추출 ---
    """

    # 버튼 클릭으로 새로 로드된 기사들은 Selenium이 로드한 DOM에만 존재하므로 request에는 안잡힘.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 뉴스 기사 링크 추출

    news_tag = soup.select('a.sa_text_title._NLOG_IMPRESSION')

    article_url_list = []

    for i in news_tag:
        href = i.get('href')
        full_url = urljoin(url, href)
        
        if href:
            article_url_list.append(full_url)

        else:
            break
    
    print("----url추출 및 list에 보냄----")

    article_url_df = pd.DataFrame(article_url_list, columns=['extracted_url'])

    return article_url_df

In [5]:
def crawl_with_selenium():

    global driver
    global data, base_url
    
    # __main__
    topic_urls = make_topic_urllist(data, base_url)
    
    article_url = [] # 모든 단일기사 url 저장 리스트

    for u in tqdm(range(len(topic_urls))):
        
        print("--토픽별 단일 기사 url 크롤링 시작---")
        article_url.append(scroll_click_extract(topic_urls[u]))
        
    return article_url

In [27]:
### Test 실행 cell

driver = None
data = './naver_topicSet3.csv'
base_url = "https://news.naver.com/breakingnews/section"
article_topic_df=[]
article_single_url_set=[]


def exec_code():
    selenium_set() 
    global article_topic_df
    global article_single_url_set
    
    article_topic_df=make_topic_urllist(data, base_url)
    article_single_url_set=crawl_with_selenium()

    return article_topic_df, article_single_url_set

In [28]:
exec_code()

  0%|          | 0/48 [00:00<?, ?it/s]

--토픽별 단일 기사 url 크롤링 시작---


  2%|▏         | 1/48 [00:02<01:39,  2.11s/it]

----url추출 및 list에 보냄----
--토픽별 단일 기사 url 크롤링 시작---


  4%|▍         | 2/48 [00:03<01:24,  1.83s/it]

...

 98%|█████████▊| 47/48 [00:58<00:01,  1.09s/it]

----url추출 및 list에 보냄----
--토픽별 단일 기사 url 크롤링 시작---


100%|██████████| 48/48 [01:00<00:00,  1.25s/it]

----url추출 및 list에 보냄----


(['https://news.naver.com/breakingnews/section/100/264',
  'https://news.naver.com/breakingnews/section/100/265',
  'https://news.naver.com/breakingnews/section/100/268',
  'https://news.naver.com/breakingnews/section/100/266',
  'https://news.naver.com/breakingnews/section/100/267',
  'https://news.naver.com/breakingnews/section/100/269',
  'https://news.naver.com/breakingnews/section/101/259',
  'https://news.naver.com/breakingnews/section/101/258',
  'https://news.naver.com/breakingnews/section/101/261',
  'https://news.naver.com/breakingnews/section/101/771',
  'https://news.naver.com/breakingnews/section/101/260',
  'https://news.naver.com/breakingnews/section/101/262',
  'https://news.naver.com/breakingnews/section/101/310',
  'https://news.naver.com/breakingnews/section/101/263',
  'https://news.naver.com/breakingnews/section/102/249',
  'https://news.naver.com/breakingnews/section/102/250',
  'https://news.naver.com/breakingnews/section/102/251',
  'https://news.naver.com/break

In [29]:
print(article_single_url_set[47].iloc[0])
pd.set_option('display.max_colwidth', None)

extracted_url    https://n.news.naver.com/mnews/article/029/0002974213
Name: 0, dtype: object
